# Analizy Przestrzenne 2022Z
# Projekt 1: Wskazanie optymalnej lokalizacji inwestycji hotelowej – analizy wielokryterialne
dane wejściowe: 
* BDOT caly
* NMT sklejony dla gminy
* gazociag
* dane glebowo-rolnicze (gleby)
* granice gminy
* działki

dane wyjściowe:
* TBA

# Import bibliotek, ustawienie środowiska, ustawienie ścieżek do warstw

In [1]:
import sys
import arcpy
import os
import math

In [2]:
# sprawdzanie rozszerzeń
# https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/checkextension.htm
arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")

'CheckedOut'

In [3]:
# nadpisywanie plików - nie działa, bo ESRI...
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

In [4]:
nazwa_gminy = "Świeradów-Zdrój"  # useless


#------------------------ ŚWIERADÓW --------------------------#
# # workspace_path = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\MyProject1.gdb"  # gdb
# # folder_bdot = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\bdot_caly"
# # path_nmt = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\nmt_gminy\nmt_swieradow.tif"
# # path_gazociag = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\gazociag\gazociag.shp"
# # path_gleby = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\gleby\gleby_gm_swieradow.shp"
# # path_granice_gminy = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\granice_gminy\gmina_swieradow.gpkg\main.gmina_swieradow"
# # path_dzialki = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\dzialki_gminy_egib\dzialki_egib.gpkg"

#------------------------ ŚWIERADÓW --------------------------#
workspace_path = r"C:\SEM5\AP\p1\analiza_hotele\analiza_hotele.gdb"  # gdb
folder_bdot = r"C:\SEM5\AP\p1\DANE\bdot_caly"
path_nmt = r"C:\SEM5\AP\p1\DANE\nmt_gminy\nmt_swieradow.tif"
path_gazociag = r"C:\SEM5\AP\p1\DANE\gazociag\gazociag.shp"
path_gleby = r"C:\SEM5\AP\p1\DANE\gleby\gleby_gm_swieradow.shp"
path_granice_gminy = r"C:\SEM5\AP\p1\DANE\granice_gminy\gmina_swieradow.gpkg\main.gmina_swieradow"
path_dzialki = r"C:\SEM5\AP\p1\DANE\dzialki_gminy_egib\dzialki_egib.shp"  # r"C:\SEM5\AP\p1\DANE\dzialki_gminy_egib\dzialki_egib.gpkg"
bufor_gminy = arcpy.analysis.Buffer(path_granice_gminy, "bufor_gminy", 100)


#------------------------ KONTROLA PLEŚNA --------------------------#
# workspace_path = r"C:\SEM5\AP\p1\analiza_hotele\analiza_hotele_kontrola.gdb"
# # folder_bdot = r"C:\SEM5\AP\p1\DANE_kontrola\bdot_caly"  # dziwny (stary?) bdot, bez potrzebnych atrybutów jak FUNSZCZ w BUBDA
# folder_bdot = r"C:\SEM5\AP\p1\TODELETE"  # dobry BDOT aktualny powiatowy
# path_nmt = r"C:\SEM5\AP\p1\DANE_kontrola\nmt_gminy\nmt_lpis_plesna_1992.tif"
# path_gazociag = r"C:\SEM5\AP\p1\DANE_kontrola\gazociag\gazociag.shp"
# path_gleby = r"C:\SEM5\AP\p1\DANE_kontrola\gleby\mapa_25000_Plesna.shp"
# path_granice_gminy = r"C:\SEM5\AP\p1\DANE_kontrola\granice_gminy\gmina_plesna.shp"
# path_dzialki = r"C:\SEM5\AP\p1\DANE_kontrola\dzialki_gminy_egib\dzialki_Plesna.shp"
# bufor_gminy = arcpy.analysis.Buffer(path_granice_gminy, "bufor_gminy", 100)


In [5]:
# ustawienia środowiska pracy
# data paths
arcpy.env.workspace = workspace_path  # geodatabase
# processing extent
arcpy.env.extent = str(bufor_gminy)  # path to bufor_gminy
# output coordinate system
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 Poland CS92")
# cell size
arcpy.env.cellSize = 1
# nadpisywanie plików
arcpy.env.overwriteOutput = True

# Ogarnięcie nazw plików z folderu BDOT i zapisanie potrzebnych ścieżek

In [6]:
# Pozbycie się kropek z nazw plików w folderze bdot_caly

bdot = dict()
names = {'budynki':'bubd_a', 'drogi':'skdr_l', 'rzeki':'swrs_l', 'jeziora':'ptwp_a', 'lasy':'ptlz_a'}
extensions = ('.shp', '.prj', '.xlsx', '.dbf', '.shx', '.cpg')
for filename in os.listdir(folder_bdot):
    if os.path.isfile(os.path.join(folder_bdot, filename)):
        src = rf"{folder_bdot}\{filename}"
        if src.endswith(extensions):
            if src.count('.') == 4:
                for k, v in names.items():
                    if v in filename.lower():
                        dst = src.replace('.', '', 3)
                        os.rename(src, dst)
                        if src.endswith('.shp'):
                            bdot[k] = dst
                if 'PT' in src:
                    dst = src.replace('.', '', 3)
                    try:
                        os.rename(src, dst)
                    except FileNotFoundError:
                        pass

In [7]:
# Kreacja słownika bdot jesli juz mamy dobre nazwy warstw

bdot = dict()
names = {'budynki':'bubd_a', 'drogi':'skdr_l', 'rzeki':'swrs_l', 'jeziora':'ptwp_a', 'lasy':'ptlz_a'}
extensions = ('.shp', '.prj', '.xlsx', '.dbf', '.shx', '.cpg')
for filename in os.listdir(folder_bdot):
    if os.path.isfile(os.path.join(folder_bdot, filename)):
        src = rf"{folder_bdot}\{filename}"
        if src.endswith(extensions):
            if src.count('.') == 1:
                for key, name in names.items():
                    if name in filename.lower() and src.endswith('.shp'):
                        bdot[key] = src

In [8]:
# bdot = {'budynki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_BUBD_A.shp', 'lasy': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTLZ_A.shp', 'jeziora': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTWP_A.shp', 'drogi': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SKDR_L.shp', 'rzeki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SWRS_L.shp'}
bdot

{'budynki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_BUBD_A.shp', 'lasy': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTLZ_A.shp', 'jeziora': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTWP_A.shp', 'drogi': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SKDR_L.shp', 'rzeki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SWRS_L.shp'}

# Kryterium 1 - odległość od konkurencji minimum 400m

In [9]:
query = "FUNSZCZ LIKE '%hotel%' OR FUNSZCZ LIKE '%motel%' OR FUNSZCZ LIKE '%pensjonat%' OR FUNSZCZ LIKE '%domWypoczynkowy%'"
hotele = arcpy.analysis.Select(bdot['budynki'], "hotele", query)
ed = arcpy.sa.EucDistance(hotele)
minimum = 400 # 400
maximum = 800 # 800

k1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

# Kryterium 2 - odległość od budynków mieszkalnych od 25 do 150m

In [10]:
query = "FUNOGBUD LIKE '%miesz%' OR FUNOGBUD LIKE '%Miesz%'"  # 1036
mieszkalne = arcpy.analysis.Select(bdot['budynki'], "mieszkalne", query)
ed = arcpy.sa.EucDistance(mieszkalne)
minimum = 25 # 25
maximum = 50 # 50
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 150 # 150
maximum = 125 # 125
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k2 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 3 - odległość od istniejących dróg od 15 do 100m

In [11]:
drogi = bdot['drogi']
ed = arcpy.sa.EucDistance(drogi)
minimum = 15 # 15
maximum = 20 # 20
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 200 # 200
maximum = 100 # 100
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k3 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 4 - odległość od rzek i zbiorników wodnych nieprzekraczalna strefa ochronna poniżej 20m

In [12]:
query = "X_KOD='SWRS01' OR X_KOD='SWRS02'"
rzeki = arcpy.analysis.Select(bdot['rzeki'], 'rzeki', query)
bufor_rzeki = arcpy.analysis.Buffer(rzeki, 'bufor_rzeki', 0.5)
jeziora = bdot['jeziora']
wody = arcpy.management.Merge([bufor_rzeki, jeziora], 'wody')
ed = arcpy.sa.EucDistance(wody)
minimum = 20 # 20
maximum = 20.001 # 25 20.001
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 1000 # 1000
maximum = 100 # 100
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k4 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 5 - pokrycie terenu nie w lesie

In [13]:
lasy_raster = arcpy.conversion.FeatureToRaster(bdot['lasy'], "RODZAJ", 'lasy_raster')

k5 = arcpy.sa.Reclassify(lasy_raster, "RODZAJ", arcpy.sa.RemapValue([["las", 0], ["zadrzewienie", 1],["zagajnik", 0], ["NODATA", 1]]))

# Kryterium 6 - nachylenie stoków maksymalnie 20%

In [14]:
slope_perc = arcpy.sa.Slope(path_nmt, "PERCENT_RISE", 1, "PLANAR", "METER")
minimum = 20 # 20
maximum = 5 # 5

k6 = arcpy.sa.FuzzyMembership(slope_perc, arcpy.sa.FuzzyLinear(minimum, maximum))

# Kryterium 7 - dostęp światła słonecznego stoki południowe (SW-SE)

In [15]:
aspect = arcpy.sa.Aspect(path_nmt)
minimum = 110 # 110
maximum = 155 # 155
fuzzy1 = arcpy.sa.FuzzyMembership(aspect, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 250 # 250 
maximum = 205 # 205
fuzzy2 = arcpy.sa.FuzzyMembership(aspect, arcpy.sa.FuzzyLinear(minimum, maximum))

k7_1 = arcpy.sa.Times(fuzzy1, fuzzy2)
k7 = arcpy.sa.Con(aspect, 1, k7_1, "VALUE = -1")

# Kryterium 8 - poza strefą ochronną gazociągu minimum 25m

In [16]:
ed = arcpy.sa.EucDistance(path_gazociag)
minimum = 25 # 25
maximum = 25.001 # 26 25.001
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 1000 # 1000
maximum = 100  # 100
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k8 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 9 - użytkowanie terenu - przydatne gleby

In [17]:
codeblock = """
def is_useful(string):
    if string in ['G','E','M','T','F','Fb','Fc','FG']:
        return "NIE"  # 0
    return "TAK"  # 1
"""
expression = "is_useful(!TYP!)"
field_name = "przydatne"
gleby = arcpy.management.CalculateField(path_gleby, field_name, expression,"PYTHON3", codeblock)

k9 = arcpy.conversion.FeatureToRaster(gleby, "przydatne", "k9")
k9 = arcpy.sa.Reclassify(k9, "przydatne", arcpy.sa.RemapValue([["NIE", 0], ["TAK", 1]]))

# Łączenie kryteriów

### Zrobienie kryteriów ostrych ze stref ochronnych wodociągu i wód

In [18]:
# 2 + 2 = 4 (5) ostrych (lasy, gleby i dodatkowo strefy ochronne wód i gazociągu)
# Weighted sum do fuzzy 7/8 k1 k2 k3 k4 k6 k7 k8
# Fuzzy overlay do ostrych 4/5 k5 k9 k4_ostre k8_ostre
# i potem mnozymy funkcją Times
# lub Raster Calculator
# dwa scenariusze - jednakowe wagi i wagi eksperckie

In [19]:
ed = arcpy.sa.EucDistance(wody)
k4_ostre = arcpy.sa.Con(ed, 1, 0, "VALUE > 20")

In [20]:
ed = arcpy.sa.EucDistance(path_gazociag)
k8_ostre = arcpy.sa.Con(ed, 1, 0, "VALUE > 25")

### Scenariusz 1: wagi jednakowe

In [21]:
wstable_1 = arcpy.sa.WSTable([[k1,"VALUE",0.1428],[k2,"VALUE",0.1428],[k3,"VALUE",0.1428],[k4,"VALUE",0.1428],[k6,"VALUE",0.1428],[k7,"VALUE",0.1428],[k8,"VALUE",0.1432]])
joined_fuzzy_1 = arcpy.sa.WeightedSum(wstable_1)

In [22]:
fo_ostre = arcpy.sa.FuzzyOverlay([k5, k9, k4_ostre, k8_ostre], "AND")  # "AND"

In [23]:
final_fuzzy_1 = arcpy.sa.Times(joined_fuzzy_1, fo_ostre)

In [24]:
ff1_reclass = arcpy.sa.Con(final_fuzzy_1, 1, 0, "VALUE > 0.5")  # VALUE > 0.5

### Scenariusz 2: wagi eksperckie

* Kryterium 1 - ```0.2``` - odległość od konkurencji minimum 400m
* Kryterium 2 - ```0.05``` - odległość od budynków mieszkalnych od 25 do 150m
* Kryterium 3 - ```0.05``` - odległość od istniejących dróg od 15 do 100m
* Kryterium 4 - ```0.2``` - odległość od rzek i zbiorników wodnych
* Kryterium 6 - ```0.15``` - nachylenie stoków maksymalnie 20%
* Kryterium 7 - ```0.3``` - dostęp światła słonecznego stoki południowe (SW-SE)
* Kryterium 8 - ```0.05``` - odległość od gazociągu

In [25]:
wstable_2 = arcpy.sa.WSTable([[k1,"VALUE",0.2],[k2,"VALUE",0.05],[k3,"VALUE",0.05],[k4,"VALUE",0.2],[k6,"VALUE",0.15],[k7,"VALUE",0.3],[k8,"VALUE",0.05]])
joined_fuzzy_2 = arcpy.sa.WeightedSum(wstable_2)

In [26]:
fo_ostre = arcpy.sa.FuzzyOverlay([k5, k9, k4_ostre, k8_ostre], "AND")  # "AND"

In [27]:
final_fuzzy_2 = arcpy.sa.Times(joined_fuzzy_2, fo_ostre)

In [28]:
ff2_reclass = arcpy.sa.Con(final_fuzzy_2, 1, 0, "VALUE > 0.5")  # VALUE > 0.5

# Kryterium 10 - przydatne działki obszary 1-6ha (mogą być też większe)

### Scenariusz 1: wagi jednakowe

In [29]:
ff1_polygons = arcpy.conversion.RasterToPolygon("ff1_reclass", "ff1_polygons", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

In [30]:
query = "gridcode = 1"
wybrane = arcpy.analysis.Select(ff1_polygons, "wybrane", query)

In [31]:
ff1_stats_table = arcpy.analysis.TabulateIntersection(path_dzialki, "ID_DZIALKI", wybrane, "ff1_stats_table", None, None, None, "UNKNOWN")  # path_table1

In [32]:
ff1_dzialki = arcpy.management.AddJoin(path_dzialki, "ID_DZIALKI", ff1_stats_table, "ID_DZIALKI", "KEEP_COMMON", "NO_INDEX_JOIN_FIELDS")

In [33]:
query = "PERCENTAGE > 70"  # >70 
ff1_dzialki_wybrane = arcpy.analysis.Select(ff1_dzialki, "ff1_dzialki_wybrane", query)

In [34]:
ff1_wybrane_dissolved = arcpy.management.Dissolve(ff1_dzialki_wybrane, "ff1_wybrane_dissolved", None, None, "SINGLE_PART", "DISSOLVE_LINES", '')

In [35]:
query = "Shape_Area > 10000"  # powierzchnia > 1ha
ff1_dissolved_duze = arcpy.analysis.Select(ff1_wybrane_dissolved, "ff1_dissolved_duze", query)

### Scenariusz 2: wagi eksperckie

In [36]:
ff2_polygons = arcpy.conversion.RasterToPolygon("ff2_reclass", "ff2_polygons", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

In [37]:
query = "gridcode = 1"
wybrane = arcpy.analysis.Select(ff2_polygons, "wybrane", query)

In [38]:
ff2_stats_table = arcpy.analysis.TabulateIntersection(path_dzialki, "ID_DZIALKI", wybrane, "ff2_stats_table", None, None, None, "UNKNOWN")  # path_table2

In [39]:
ff2_dzialki = arcpy.management.AddJoin(path_dzialki, "ID_DZIALKI", ff2_stats_table, "ID_DZIALKI", "KEEP_COMMON", "NO_INDEX_JOIN_FIELDS")

In [40]:
query = "PERCENTAGE > 70"  # >70 
ff2_dzialki_wybrane = arcpy.analysis.Select(ff2_dzialki, "ff2_dzialki_wybrane", query)

In [41]:
ff2_wybrane_dissolved = arcpy.management.Dissolve(ff2_dzialki_wybrane, "ff2_wybrane_dissolved", None, None, "SINGLE_PART", "DISSOLVE_LINES", '')

In [42]:
query = "Shape_Area > 10000"  # powierzchnia > 1ha
ff2_dissolved_duze = arcpy.analysis.Select(ff2_wybrane_dissolved, "ff2_dissolved_duze", query)

# Kryterium 11 - ostateczne v1 - kształt obszaru jak najbardziej zwarty

### Scenariusz 1: wagi jednakowe

C (compactness) = pierwiastek z pola obiektu dzielonego przez pole koła. Promień do wzoru na pole koła należy wziąć z obwodu koła, które jest równe obwodowi obiektu.

In [43]:
codeblock = """
def compactness(pole_obiektu, obwod_obiektu):
    # pole = pi * r**2
    # obwod = 2*pi*r
    r = obwod_obiektu / (2*math.pi)
    pole_kola = math.pi * r**2
    c = math.sqrt(pole_obiektu / pole_kola)
    return c
"""
expression = "compactness(!Shape_Area!, !Shape_Length!)"
field_name = "compactness"
ff1_compactness = arcpy.management.CalculateField(ff1_dissolved_duze, field_name, expression, "PYTHON3", codeblock)

In [44]:
# znalezienie najbardziej zwartego obszaru 
compactness = {key:value for (key, value) in arcpy.da.SearchCursor(ff1_compactness,['OID@','compactness'])}
max_oid, max_compactness = max(compactness.items(), key=lambda krotka: krotka[1])
print(max_oid, max_compactness)

24 0,893322519992807


In [45]:
query = f"OBJECTID IN ({max_oid})"
ff1_best = arcpy.analysis.Select(ff1_compactness, "ff1_best", query)

### Scenariusz 2: wagi eksperckie

In [46]:
codeblock = """
def compactness(pole_obiektu, obwod_obiektu):
    # pole = pi * r**2
    # obwod = 2*pi*r
    r = obwod_obiektu / (2*math.pi)
    pole_kola = math.pi * r**2
    c = math.sqrt(pole_obiektu / pole_kola)
    return c
"""
expression = "compactness(!Shape_Area!, !Shape_Length!)"
field_name = "compactness"
ff2_compactness = arcpy.management.CalculateField(ff2_dissolved_duze, field_name, expression, "PYTHON3", codeblock)

In [47]:
# znalezienie najbardziej zwartego obszaru 
compactness = {key:value for (key, value) in arcpy.da.SearchCursor(ff2_compactness,['OID@','compactness'])}
max_oid, max_compactness = max(compactness.items(), key=lambda krotka: krotka[1])
print(max_oid, max_compactness)

8 0,885187667430222


In [48]:
query = f"OBJECTID IN ({max_oid})"
ff2_best = arcpy.analysis.Select(ff2_compactness, "ff2_best", query)

# Kryterium 12 - ostateczne v2 - wskazanie optymalnej lokalizacji z wykorzystaniem map kosztów

### Stworzenie mapy kosztów

In [49]:
layers_to_union = list()
for filename in os.listdir(folder_bdot):
    if os.path.isfile(os.path.join(folder_bdot, filename)):
        src = rf"{folder_bdot}\{filename}"
        if 'PT' in src and src.endswith('.shp'):
            layers_to_union.append(src)
#             print(src)

In [50]:
PT_union = arcpy.management.Merge(layers_to_union, "PT_union", None, "NO_SOURCE_INFO")

In [51]:
arcpy.management.AddField(PT_union, "Value_v2", "SHORT", None, None, None, '',"NULLABLE", "NON_REQUIRED", '')

<Result 'C:\\SEM5\\AP\\p1\\analiza_hotele\\analiza_hotele.gdb\\PT_union'>

In [52]:
costs_dict = {
"PTWP01": 0,
"PTWP02": 200,
"PTWP03": 0,
"PTZB01": 200,
"PTZB02": 100,
"PTZB03": 200,
"PTZB04": 200,
"PTZB05": 50,
"PTLZ01": 100,
"PTLZ02": 50,
"PTLZ03": 50,
"PTRK": 15,
"PTRK02": 15,
"PTUT01": 0,
"PTUT02": 90,
"PTUT03": 100,
"PTUT04": 20,
"PTUT05": 20,
"PTTR01": 20,
"PTTR02": 1,
"PTKM01": 100,
"PTKM02": 200,
"PTKM03": 170,
"PTKM04": 200,
"PTGN01": 1,
"PTGN02": 1,
"PTGN03": 1,
"PTGN04": 1,
"PTPL01": 50,
"PTSO01": 0,
"PTSO02": 0,
"PTWZ01": 0,
"PTWZ02": 0,
"PTNZ01": 150,
"PTNZ02": 150}

In [53]:
codeblock = """
def xkod_to_cost(x_kod):
    costs_dict = {
    "PTWP01": 0,
    "PTWP02": 200,
    "PTWP03": 0,
    "PTZB01": 200,
    "PTZB02": 100,
    "PTZB03": 200,
    "PTZB04": 200,
    "PTZB05": 50,
    "PTLZ01": 100,
    "PTLZ02": 50,
    "PTLZ03": 50,
    "PTRK": 15,
    "PTRK02": 15,
    "PTUT01": 0,
    "PTUT02": 90,
    "PTUT03": 100,
    "PTUT04": 20,
    "PTUT05": 20,
    "PTTR01": 20,
    "PTTR02": 1,
    "PTKM01": 100,
    "PTKM02": 200,
    "PTKM03": 170,
    "PTKM04": 200,
    "PTGN01": 1,
    "PTGN02": 1,
    "PTGN03": 1,
    "PTGN04": 1,
    "PTPL01": 50,
    "PTSO01": 0,
    "PTSO02": 0,
    "PTWZ01": 0,
    "PTWZ02": 0,
    "PTNZ01": 150,
    "PTNZ02": 150
    }
    cost = costs_dict[x_kod]
    return cost
"""
expression = "xkod_to_cost(!X_KOD!)"
field_name = "Value_v2"
PT_union = arcpy.management.CalculateField(PT_union, field_name, expression, "PYTHON3", codeblock)

In [54]:
PT_union_raster = arcpy.conversion.FeatureToRaster(PT_union, "Value_v2", "PT_union_raster", 1)

In [55]:
mapa_kosztow = arcpy.sa.SetNull(PT_union_raster, PT_union_raster, "Value = 0")

### Scenariusz 1: wagi jednakowe

In [56]:
koszty_skumulowane_1 = arcpy.sa.CostDistance(ff1_dissolved_duze, mapa_kosztow, None, "kierunki", None, None, None, None, '')

In [57]:
ff1_best_gazociag = arcpy.sa.CostPath(path_gazociag, koszty_skumulowane_1, "kierunki", "BEST_SINGLE")

In [58]:
# ff1_best_gazociag = arcpy.conversion.RasterToPolygon(ff1_best_gazociag, "ff1_best_gazociag", "SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
# ff1_best_cost = arcpy.management.SelectLayerByLocation(ff1_dissolved_duze, "INTERSECT", ff1_best_gazociag, None, "NEW_SELECTION", "NOT_INVERT")
# ff1_best_gazociag = arcpy.management.FeatureToLine(ff1_best_gazociag, "ff1_best_gazociag_line", None, "ATTRIBUTES")

### Scenariusz 2: wagi eksperckie

In [59]:
koszty_skumulowane_2 = arcpy.sa.CostDistance(ff2_dissolved_duze, mapa_kosztow, None, "kierunki", None, None, None, None, '')

In [60]:
ff2_best_gazociag = arcpy.sa.CostPath(path_gazociag, koszty_skumulowane_2, "kierunki", "BEST_SINGLE")

# TRASH

In [61]:
# ?arcpy.sa.CostPath